## 데이터프레임 합치기
- 두개 이상의 DataFrame을 합쳐 하나의 DataFrame으로 만든다.
   
## 데이터셋 읽기
- stocks_2016.csv, stocks_2017.csv, stocks_2018.csv : 년도별 보유 주식
- stocks_info.csv : 주식 정보

In [5]:
import pandas as pd

s_2016 = pd.read_csv('data/stocks_2016.csv')
s_2017 = pd.read_csv('data/stocks_2017.csv')
s_2018 = pd.read_csv('data/stocks_2018.csv')
s_info = pd.read_csv('data/stocks_info.csv')

In [6]:
f_txt = ['2016', '2017', '2018', 'info']
s_2016, s_2017, s_2018, s_info = [pd.read_csv(f'data/stocks_{txt}.csv') for txt in f_txt]

print(s_2016.shape, s_2017.shape, s_2018.shape, s_info.shape)

# for t in f_txt:
#     pd.read_csv(f'data/stocks_{t}.csv')

(3, 4) (6, 4) (3, 4) (8, 2)


## concat() 이용
- 수직, 조인을 이용한 수평 결합 모두 지원한다.
- 수직 결합의 경우 합치는 기준: 컬럼명이 같은 열끼리 합친다.
- 조인(수평결함)의 경우 full outer join과 inner join을 지원한다.
    - full outer join이 기본 방식
    - 조인 기준: index가 같은 행 끼리 합친다. (equi-join)
- pd.concat(objs,  [, key=리스트]), axis=0, join='outer' )
    - 매개변수
        - objs: 합칠 DataFrame들을 리스트로 전달
        - keys=[] 를 이용해 합친 행들을 구분하기 위한 다중 인덱스 처리
        - axis
            - 0 또는 index : 수직결합
            - 1 또는 columns : 수평결합
        - join: 
            - 조인방식
            - 'outer'(기본값) 또는 'inner'

#### concat()

In [7]:
df = pd.concat([s_2016, s_2017, s_2018])
print(df.shape)
df

(12, 4)


,Symbol,Shares,Low,High
0,AAPL,80,95,110
1,TSLA,50,80,130
2,WMT,40,55,70
0,AAPL,50,120,140
1,GE,100,30,40
2,IBM,87,75,95
3,SLB,20,55,85
4,TXN,500,15,23
5,TSLA,100,100,300
0,AAPL,40,135,170


#### 합칠때 index는 버리기 => 행식별

In [10]:
df2 = pd.concat([s_2016, s_2017, s_2018], ignore_index=True)
df2

,Symbol,Shares,Low,High
0,AAPL,80,95,110
1,TSLA,50,80,130
2,WMT,40,55,70
3,AAPL,50,120,140
4,GE,100,30,40
5,IBM,87,75,95
6,SLB,20,55,85
7,TXN,500,15,23
8,TSLA,100,100,300
9,AAPL,40,135,170


#### keys: multi-index

In [11]:
df3 = pd.concat([s_2016, s_2017, s_2018], keys=[2016, 2017, 2018])
df3

Symbol  Shares  Low  High
2016 0   AAPL      80   95   110
     1   TSLA      50   80   130
     2    WMT      40   55    70
2017 0   AAPL      50  120   140
     1     GE     100   30    40
     2    IBM      87   75    95
     3    SLB      20   55    85
     4    TXN     500   15    23
     5   TSLA     100  100   300
2018 0   AAPL      40  135   170
     1   AMZN       8  900  1125
     2   TSLA      50  220   400

#### 수평결합(axis=1) - index가 같은 행끼리 합친다. (full outer join 기본)

In [12]:
# 수평결합(axis=1) - index가 같은 행끼리 합친다. (full outer join 기본)
df4 = pd.concat([s_2016, s_2018], axis=1)
print(df4.shape)
df4

(3, 8)


,Symbol,Shares,Low,High,Symbol,Shares,Low,High
0,AAPL,80,95,110,AAPL,40,135,170
1,TSLA,50,80,130,AMZN,8,900,1125
2,WMT,40,55,70,TSLA,50,220,400


## 조인을 통한 DataFrame 합치기
- join()
    - 2개 이상의 DataFrame을 조인할 때 사용
- merge()
    - 2개의 DataFrame의 조인만 지원

### join()
- dataframe객체.join(others, how='left', lsuffix='', rsuffix='') 
- `df_A.join(df_b)`, `df_A.join([df_b, df_c, df_d])`
- 두개 이상의 DataFrame들을 조인 할 수 있다.
    - **조인 기준**: index가 같은 값인 행끼리 합친다. (equi-join)
    - **조인 기본 방식**: Left Outer Join
- 매개변수
    - lsuffix, rsuffix
        - 조인 대상 DataFrame에 같은 이름의 컬럼이 있으면 에러 발생.
        - 같은 이름이 있는 경우 붙일 접미어 지정
    - how :조인방식. 'left', 'right', 'outer', 'inner'. left가 기본
> - 여러 데이터프레임에 흩어져 있는 정보 중 필요한 정보만 모아서 결합하기 위한 것.
> - 두개 이상의 데이터프레임을 특정 컬럼(열)의 값이 같은 행 끼리 수평 결합하는 것.
> - Inner Join, Left Outer Join, Right Outer Join, Full Outer Join       

#### lsuffix,rsuffix
- 컬럼명에 붙일 접미어 설정

In [13]:
# target(s_info), source(s_2017)
# s_info.join(s_2017)  #join 대상 dataframe에 같은 이름의 컬럼이 있으면  Error
s_info.join(s_2017, lsuffix="_info", rsuffix='_2017')
# index명이 같은 행끼리 합친다. (s_info, s_2017 데이터셋에 맞춰서는 잘못 join이 됨.)

,Symbol_info,Name,Symbol_2017,Shares,Low,High
0,AAPL,Apple Inc,AAPL,50.0,120.0,140.0
1,TSLA,Tesla Inc,GE,100.0,30.0,40.0
2,WMT,Walmart Inc,IBM,87.0,75.0,95.0
3,GE,General Electric,SLB,20.0,55.0,85.0
4,IBM,IBM(International Business Machines Co),TXN,500.0,15.0,23.0
5,SLB,Schlumberger Limited.,TSLA,100.0,100.0,300.0
6,TXN,Texas Instruments Incorporated,NaN,NaN,NaN,NaN
7,AMZN,"Amazon.com, Inc",NaN,NaN,NaN,NaN


#### 같은 컬럼이 있을경우 index 로 빼주어 join

In [14]:
s_info.set_index('Symbol').join(s_2017.set_index("Symbol"), how='left')  #how=inner (inner join)

,Name,Shares,Low,High
Symbol,,,,
AAPL,Apple Inc,50.0,120.0,140.0
TSLA,Tesla Inc,100.0,100.0,300.0
WMT,Walmart Inc,NaN,NaN,NaN
GE,General Electric,100.0,30.0,40.0
IBM,IBM(International Business Machines Co),87.0,75.0,95.0
SLB,Schlumberger Limited.,20.0,55.0,85.0
TXN,Texas Instruments Incorporated,500.0,15.0,23.0
AMZN,"Amazon.com, Inc",NaN,NaN,NaN


#### add_suffix / add_prefix
- 모든 컬럼명 뒤/앞에 지정한 문자열을 붙여준다

In [16]:
s_info_2 = s_info.copy()
s_2016_2 = s_2016.copy()
s_2017_2 = s_2017.copy()

In [17]:
others = [
    s_2016_2.set_index('Symbol').add_suffix('_2016'),
    s_2017_2.set_index('Symbol').add_suffix('_2017'),
    s_2018.set_index('Symbol').add_suffix('_2018')
]

s_info_2.set_index('Symbol').join(others)

,Name,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017,Shares_2018,Low_2018,High_2018
Symbol,,,,,,,,,,
AAPL,Apple Inc,80.0,95.0,110.0,50.0,120.0,140.0,40.0,135.0,170.0
TSLA,Tesla Inc,50.0,80.0,130.0,100.0,100.0,300.0,50.0,220.0,400.0
WMT,Walmart Inc,40.0,55.0,70.0,NaN,NaN,NaN,NaN,NaN,NaN
GE,General Electric,NaN,NaN,NaN,100.0,30.0,40.0,NaN,NaN,NaN
IBM,IBM(International Business Machines Co),NaN,NaN,NaN,87.0,75.0,95.0,NaN,NaN,NaN
SLB,Schlumberger Limited.,NaN,NaN,NaN,20.0,55.0,85.0,NaN,NaN,NaN
TXN,Texas Instruments Incorporated,NaN,NaN,NaN,500.0,15.0,23.0,NaN,NaN,NaN
AMZN,"Amazon.com, Inc",NaN,NaN,NaN,NaN,NaN,NaN,8.0,900.0,1125.0


### merge()
- `df_a.merge(df_b)`
- 두개의 DataFrame 조인만 지원
    - **조인 기준**: 같은 컬럼명을 기준으로 equi-join이 기본. **조인기준을 다양하게 정할 수 있다.**
    - **조인 기본 방식**: inner join
- `dataframe.merge(합칠dataframe, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False)`  
- 매개변수
    - on : 같은 컬럼명이 여러개일때 join 대상 컬럼을 선택
    - right_on, left_on : 조인할 때 사용할 왼쪽,오른쪽 Dataframe의 컬럼명. 
    - left_index, right_index: 조인 할때 index를 사용할 경우 True로 지정 
    - how : 조인 방식.  'left', 'right', 'outer', 'inner'. 기본: inner 
    - suffixes: 두 DataFrame에 같은 이름의 컬럼명이 있을 경우 구분을 위해 붙인 접미어를 리스트로 설정
        - 생략시 x, y를 붙인다.       

#### 기본방식: inner join

In [18]:
s_2016.merge(s_2017, on='Symbol') #양쪽의 Symbol 컬럼 값이 같은 행끼리 join
# join 대상 컬럼은 하나만 나온다.
# 그 이외 컬럼들은 모두 나온다. 단 중복된 이름의 컬럼의 경우 _x, _y suffix를 붙여준다.

,Symbol,Shares_x,Low_x,High_x,Shares_y,Low_y,High_y
0,AAPL,80,95,110,50,120,140
1,TSLA,50,80,130,100,100,300


#### outer join

In [19]:
s_2016.merge(s_2017, on='Symbol', suffixes=['_2016', '_2017'], how='outer') #full outer join (s_2016, s_2017 모두다 나온다.)

,Symbol,Shares_2016,Low_2016,High_2016,Shares_2017,Low_2017,High_2017
0,AAPL,80.0,95.0,110.0,50.0,120.0,140.0
1,TSLA,50.0,80.0,130.0,100.0,100.0,300.0
2,WMT,40.0,55.0,70.0,NaN,NaN,NaN
3,GE,NaN,NaN,NaN,100.0,30.0,40.0
4,IBM,NaN,NaN,NaN,87.0,75.0,95.0
5,SLB,NaN,NaN,NaN,20.0,55.0,85.0
6,TXN,NaN,NaN,NaN,500.0,15.0,23.0


In [23]:
# s_2016 - Symbol 컬럼, s_info_2 - index명  을 기준으로 join
s_2016.merge(s_info_2.set_index('Symbol'),
             left_on="Symbol", # 왼쪽 DataFrame의 join 연산 기준 - 컬럼: left_on
             right_index=True  # 오른쪽 DataFrame의 join 연산 기준 - index: right_index
            ) #왼쪽(s_2016)의 Symbol 컬럼과 오른쪽(s_info_2)의 index명 이 같은 행끼리 join. 방식: inner(default)

,Symbol,Shares,Low,High,Name
0,AAPL,80,95,110,Apple Inc
1,TSLA,50,80,130,Tesla Inc
2,WMT,40,55,70,Walmart Inc


## Database Table의 데이터를 읽어서 DataFrame 생성

- pd.read_sql('select문', connection)
    - 'select문' 조회결과를 가지는 DataFrame을 반환

In [25]:
import pymysql 
import pandas as pd

In [27]:
connection = pymysql.connect(host='127.0.0.1', port=3306, user='scott', password='tiger', database='hr_join')

In [28]:
emp_test = pd.read_sql('select emp_id ID, emp_name 이름, hire_date 입사일 from emp where comm_pct is not null', connection)
emp_test

E:\anaconda3\envs\ml\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,ID,이름,입사일
0,145,John,2004-10-01
1,146,Karen,2004-10-01
2,147,Alberto,2005-03-10
3,148,Gerald,2007-10-15
4,149,Eleni,2007-10-15
5,150,Peter,2007-10-15
6,151,David,2005-03-24
7,152,Peter,2005-08-20
8,153,Christopher,2006-03-30
9,154,Nanette,2006-12-09


In [29]:
emp_dept = pd.read_sql('select  e.*, d.dept_name, d.loc from    emp e left join dept d on e.dept_id = d.dept_id', connection)
emp_dept

E:\anaconda3\envs\ml\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,emp_id,emp_name,job_id,mgr_id,hire_date,salary,comm_pct,dept_id,dept_name,loc
0,100,Steven,AD_PRES,NaN,2003-06-17,24000.0,NaN,90.0,Executive,Seattle
1,101,Neena,AD_VP,100.0,2005-09-21,17000.0,NaN,90.0,Executive,Seattle
2,102,Lex,AD_VP,100.0,2001-01-13,17000.0,NaN,90.0,Executive,Seattle
3,103,Alexander,IT_PROG,102.0,2006-01-03,9000.0,NaN,60.0,IT,San Francisco
4,104,Bruce,IT_PROG,103.0,2007-05-21,6000.0,NaN,60.0,IT,San Francisco
...,...,...,...,...,...,...,...,...,...,...
102,202,Pat,MK_REP,201.0,2005-08-17,6000.0,NaN,20.0,Marketing,New York
103,203,Susan,HR_REP,101.0,2002-06-07,6500.0,NaN,40.0,Human Resources,New York
104,204,Hermann,PR_REP,101.0,2002-06-07,10000.0,NaN,70.0,Public Relations,New York
105,205,Shelley,AC_MGR,101.0,2002-06-07,12008.0,NaN,110.0,Accounting,Seattle
